In [2]:
from pycelonis import get_celonis
from PYCELONIS.configurations import *
from pycelonis import pql
import pandas as pd
import numpy
import datetime


class Celonis:
    def __init__(self):

        self.url = load_property('CELONISCONNECTION', 'celonis_url')
        self.api_key = load_property('CELONISCONNECTION', 'celonis_api_key')
        self.celonis = get_celonis(url=self.url, api_token=self.api_key, key_type="APP_KEY")
        self.data_pool = "SAP ECC O2C"#"e8abd6b8-f2c7-4612-8578-e96b13dd8802"
        self.data_model = "O2C | DataFlow"#"db06d4e5-51e3-49fb-8da9-4a35ded7f943"


    def print_datamodel_tables(self):
        dm = self.celonis.datamodels.find(self.data_model)
        return(dm.tables)
    

    def get_data(self, table_name):
        #This method takes a table -tabname parameter- from a celonis data model.
        #query += pql.PQLFilter("tab.colname == value")
        datamodel = self.celonis.datamodels.find(self.data_model)
        columns = datamodel.tables.find(table_name).columns
        q = pql.PQL()
        for cols in range(len(columns)):
            q += pql.PQLColumn(query = table_name+ f".{columns[cols]['name']}", name =  columns[cols]['name'].upper())#.replace("/","").replace(" ","").replace("_",""))
        DF = datamodel._get_data_frame(q, chunksize = 1000000)
        return(DF)
    
    def get_data_id(self, table_name):
        datamodel = self.celonis.datamodels.find(self.data_model)
        table = datamodel.tables.find(table_name)
        columns = datamodel.tables.find(table_name).columns
        q = pql.PQL()
        for cols in range(len(columns)):
            q += pql.PQLColumn(query = table.name + f".{columns[cols]['name']}", name =  columns[cols]['name'].upper())
        DF = datamodel._get_data_frame(q, chunksize = 1000000)
        return(DF)


   #def get_data_array_OLD(self, array):
   #    datamodel = self.celonis.datamodels.find(self.data_model)
   #    col_array = array
   #    q = pql.PQL()
   #    for cols in range(len(col_array)):
   #        q += pql.PQLColumn( query= col_array[cols], name = f"{col_array[cols].split('.')[1]}")
   #    DF = datamodel._get_data_frame(q,chunksize = 1000000)
   #    return(DF)
   #

    def get_data_array(self, array):
        datamodel = self.celonis.datamodels.find(self.data_model)
        col_array = array
        q = pql.PQL()
        for cols in range(len(col_array)):
            q += pql.PQLColumn(query= col_array[cols], name = f"{col_array[cols]}")
        DF = datamodel._get_data_frame(q,chunksize = 1000000)
        return(DF)
    
    def get_cols(self,array):
        all_cols = []
        datamodel = self.celonis.datamodels.find(self.data_model)
        for i in array:
            data = datamodel.tables.find(i).columns
            cols = numpy.array([d['name'] for d in data])
            all_cols.append(cols)

        return(all_cols)
    
    def get_data_array_filter(self, array, filter):
        datamodel = self.celonis.datamodels.find(self.data_model)
        col_array = array
        q = pql.PQL()
        for cols in range(len(col_array)):
            q += pql.PQLColumn( query= col_array[cols], name = f"{col_array[cols]}") # .split('.')[1]}")
        q += pql.PQLFilter(filter)
        DF = datamodel._get_data_frame(q,chunksize = 1000000)
        return(DF)
    
    def get_cols_dict(self,array):
        cols = {}
        datamodel = self.celonis.datamodels.find(self.data_model)
        for i in array:
            data = datamodel.tables.find(i).columns
            col_names = [d['name'] for d in data]
            cols[i] = col_names

        return(cols)
    
c = Celonis()

RuntimeError: ModuleWatchdog is already installed

In [ ]:
vbap_array = ['VBAP.MANDT',"VBAP.ERDAT","VBAP.ERZET","VBAP.POSNR","VBAP.ERNAM","VBAP.NETWR","VBAP.MATNR","VBAP.WERKS","VBAP.ABGRU","VBAP.VBELN"]
vbak_array = ["VBAK.MANDT","VBAK.VBELN","VBAK.VBTYP","VBAK.OBJNR","VBAK.WAERK","VBAK.BUKRS_VF","VBAK.KUNNR","VBAK.KKBER","VBAK.LIFSK","VBAK.FAKSK"]
USR02_array = ['USR02.MANDT','USR02.BNAME','USR02.USTYP']
CDPOS_array = ['CDPOS.MANDANT','CDPOS.TABKEY',"CDPOS.OBJECTCLAS",'CDPOS.OBJECTID','CDPOS.CHANGENR','CDPOS.TABNAME','CDPOS.FNAME','CDPOS.CHNGIND', 'CDPOS.VALUE_OLD', 'CDPOS.VALUE_NEW']
CDHDR_array = ['CDHDR.MANDANT','CDHDR.TCODE',"CDHDR.OBJECTCLAS", "CDHDR.OBJECTID","CDHDR.CHANGENR", "CDHDR.UDATE", "CDHDR.UTIME", "CDHDR.USERNAME"]

In [ ]:
vbap = c.get_data_array(vbap_array)
vbap['VBAP.ERDAT'] = pd.to_datetime(vbap['VBAP.ERDAT'])  # Convert 'ERDAT' column to datetime
vbap = vbap[(vbap['VBAP.ERDAT'] <= '2021-01-05') & (vbap['VBAP.ERDAT'] >= '2021-01-01')]

In [ ]:
vbak = c.get_data_array(vbak_array)

In [ ]:
VBAK_VBAP = pd.merge(
    vbak[['VBAK.MANDT', 'VBAK.VBELN', 'VBAK.VBTYP', 'VBAK.OBJNR', 'VBAK.WAERK', 'VBAK.BUKRS_VF', 'VBAK.KUNNR', 'VBAK.KKBER', 'VBAK.LIFSK', 'VBAK.FAKSK']],
    vbap[['VBAP.MANDT', 'VBAP.VBELN', 'VBAP.ERDAT', 'VBAP.ERZET', 'VBAP.POSNR', 'VBAP.ERNAM', 'VBAP.NETWR', 'VBAP.MATNR', 'VBAP.WERKS', 'VBAP.ABGRU']],
    how='inner',
    left_on=['VBAK.MANDT', 'VBAK.VBELN'],
    right_on=['VBAP.MANDT', 'VBAP.VBELN']
).drop_duplicates()

In [ ]:
VBAK_VBAP['VBAK_VBAP._CASE_KEY'] = VBAK_VBAP['VBAK.MANDT'] + VBAK_VBAP['VBAK.VBELN'] + VBAK_VBAP['VBAP.POSNR']
VBAK_VBAP['VBAK_VBAP.TABKEY_VBAK'] = VBAK_VBAP['VBAK.MANDT'] + VBAK_VBAP['VBAK.VBELN']
VBAK_VBAP['VBAK_VBAP.EVENTTIME']= pd.to_datetime(VBAK_VBAP["VBAP.ERDAT"].dt.date.astype(str) + ' ' + VBAK_VBAP["VBAP.ERZET"].dt.strftime('%H:%M:%S'))

In [ ]:
USR02 = c.get_data_array(USR02_array)

In [ ]:
CDPOS =  c.get_data_array_filter(CDPOS_array,filter="(CDPOS.TABNAME IN ('VBAK','VBAP','FPLT')) AND (CDPOS.MANDANT in ('000'))")

In [ ]:
CDHDR =  c.get_data_array_filter(CDHDR_array,filter="CDHDR.MANDANT IN ('000')")

In [ ]:
CDPOS_CDHDR = pd.merge(
    CDHDR[['CDHDR.MANDANT', 'CDHDR.OBJECTCLAS', 'CDHDR.OBJECTID', 'CDHDR.CHANGENR', 'CDHDR.UDATE', 'CDHDR.USERNAME',"CDHDR.TCODE","CDHDR.UTIME"]],
    CDPOS[['CDPOS.MANDANT', 'CDPOS.OBJECTCLAS','CDPOS.OBJECTID','CDPOS.CHANGENR','CDPOS.FNAME', 'CDPOS.TABNAME', 'CDPOS.TABKEY', 'CDPOS.VALUE_OLD', 'CDPOS.VALUE_NEW', 'CDPOS.CHNGIND']],
    how='inner',
    left_on=['CDHDR.MANDANT', 'CDHDR.OBJECTCLAS', 'CDHDR.OBJECTID', 'CDHDR.CHANGENR'],
    right_on=['CDPOS.MANDANT', 'CDPOS.OBJECTCLAS','CDPOS.OBJECTID','CDPOS.CHANGENR']
)

#CDPOS_CDHDR = CDPOS_CDHDR[
#    (CDPOS_CDHDR['CHNGIND'] == 'U') &
#    (CDPOS_CDHDR['UDATE'].notnull()) &
#    (CDPOS_CDHDR['TABNAME'].isin(['VBAK', 'VBAP', 'FPLT']))
#]
#
CDPOS_CDHDR['CDPOS_CDHDR.EVENTTIME'] = pd.to_datetime(CDPOS_CDHDR['CDHDR.UDATE']) + pd.to_timedelta(CDPOS_CDHDR['CDHDR.UTIME'].astype(str))

#TMP_O2C_CDHDR_CDPOS = TMP_O2C_CDHDR_CDPOS[['MANDANT', 'FNAME', 'TABNAME', 'TABKEY', 'VALUE_OLD', 'VALUE_NEW', 'EVENTTIME', 'CHANGENR', 'TCODE', 'OBJECTCLAS', 'OBJECTID', 'USERNAME']]

MILESTONES

In [ ]:
create_so = pd.merge(
    VBAK_VBAP[['VBAK_VBAP._CASE_KEY', 'VBAK.MANDT', 'VBAK.VBELN', 'VBAP.POSNR','VBAP.ERDAT','VBAP.ERZET','VBAP.ERNAM']],
    USR02[['USR02.MANDT', 'USR02.BNAME', 'USR02.USTYP']],
    how='left',
    left_on=['VBAK.MANDT', 'VBAP.ERNAM'],
    right_on=['USR02.MANDT', 'USR02.BNAME']
)        


create_so['create_so.ACTIVITY_EN'] = 'Create Sales Order'
create_so['create_so._SORTING'] = 11
create_so['create_so._ACTIVITY_KEY'] = create_so['VBAK.MANDT'] + create_so['VBAK.VBELN']
create_so['create_so.EVENTTIME'] = create_so.apply(lambda row: datetime.datetime.combine(row['VBAP.ERDAT'].date(), row['VBAP.ERZET'].time()), axis=1)


In [ ]:
create_so_item = pd.merge(
    pd.merge(
        VBAK_VBAP[['VBAK_VBAP._CASE_KEY', 'VBAK.MANDT', 'VBAP.VBELN', 'VBAP.POSNR','VBAP.ERDAT','VBAP.ERZET','VBAP.ERNAM']],
        USR02[['USR02.MANDT', 'USR02.BNAME', 'USR02.USTYP']],
        how='left',
        left_on=['VBAK.MANDT', 'VBAP.ERNAM'],
        right_on=['USR02.MANDT', 'USR02.BNAME']
    ),
    CDPOS_CDHDR.groupby('CDPOS.TABKEY')['CDHDR.TCODE'].max().reset_index(),
    how='left',
    left_on=['VBAK_VBAP._CASE_KEY'],
    right_on=['CDPOS.TABKEY']
)

create_so_item['create_so_item.ACTIVITY_DE'] = 'Lege Auftragsposition an'
create_so_item['create_so_item.ACTIVITY_EN'] = 'CreateSOItem'
create_so_item['create_so_item.EVENTTIME'] = create_so_item.apply(lambda row: datetime.datetime.combine(row['VBAP.ERDAT'].date(), row['VBAP.ERZET'].time()), axis=1)
create_so_item['create_so_item._SORTING'] = 19
create_so_item['create_so_item._ACTIVITY_KEY'] = create_so_item['VBAK.MANDT'] + create_so_item['VBAP.VBELN'] + create_so_item['VBAP.POSNR']

In [ ]:
add_item = pd.merge(
    pd.merge(
        pd.merge(
            VBAK_VBAP[['VBAK_VBAP._CASE_KEY', 'VBAK.MANDT', 'VBAP.VBELN', 'VBAP.POSNR']],
            CDPOS[['CDPOS.MANDANT',"CDPOS.OBJECTCLAS","CDPOS.OBJECTID","CDPOS.CHANGENR",'CDPOS.TABKEY', 'CDPOS.FNAME', 'CDPOS.CHNGIND','CDPOS.TABNAME']],
            how='inner',
            left_on='VBAK_VBAP._CASE_KEY',
            right_on='CDPOS.TABKEY'
        ),
        CDHDR[['CDHDR.MANDANT', 'CDHDR.OBJECTCLAS', 'CDHDR.OBJECTID', 'CDHDR.CHANGENR', 'CDHDR.TCODE', 'CDHDR.USERNAME']],
        how='inner',
        left_on=['CDPOS.MANDANT', 'CDPOS.OBJECTCLAS', 'CDPOS.OBJECTID', 'CDPOS.CHANGENR'],
        right_on=['CDHDR.MANDANT', 'CDHDR.OBJECTCLAS', 'CDHDR.OBJECTID', 'CDHDR.CHANGENR']
    ),
    USR02[['USR02.MANDT', 'USR02.BNAME', 'USR02.USTYP']],
    how='left',
    left_on=['CDPOS.MANDANT', 'CDHDR.USERNAME'],
    right_on=['USR02.MANDT', 'USR02.BNAME']
)

add_item['add_item.ACTIVITY_EN'] = add_item.apply(lambda row: 'Change: Item add' if row['CDPOS.FNAME'] == 'KEY' and row['CDPOS.CHNGIND'] == 'I' else None, axis=1)
add_item['add_item._ACTIVITY_KEY'] = add_item['CDPOS.MANDANT'] + add_item['CDPOS.OBJECTCLAS'] + add_item['CDPOS.OBJECTID'] + add_item['CDPOS.CHANGENR']

Intermediary Steps

In [ ]:
milestones = [create_so, create_so_item,add_item]#ship_to_party,set_billing_block,delivery_docs,changes_schedule_line,changes_business_data,changes_so_item,changes_so_header]

In [ ]:
def build_key_value_array(dictionary):
    key_value_array = []
    
    for key, value in dictionary.items():
        for col in value:
            key_value_array.append(key + "." + col)
    
    return key_value_array

In [ ]:
list_all_tables = []
for i in c.print_datamodel_tables():
    list_all_tables.append(i.name)

all_columns = c.get_cols_dict(list_all_tables)
full_column_array = build_key_value_array(all_columns)
full_column_array

Imports

In [ ]:
from jmetal.algorithm.singleobjective.genetic_algorithm import GeneticAlgorithm 
from jmetal.operator import BitFlipMutation
from jmetal.operator.crossover import SPXCrossover
from jmetal.operator.selection import BestSolutionSelection
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.problem.singleobjective.orderToCash import orderToCash

Algorithms

In [ ]:
problem = orderToCash(milestones_test=milestones,full_datamodel_cols=full_column_array,number_of_variables=len(full_column_array), number_of_objectives=1)


algorithm = GeneticAlgorithm(
    problem = problem,
    population_size = 20,
    offspring_population_size = 20,
    mutation = BitFlipMutation(probability=1/problem.number_of_variables),
    crossover=  SPXCrossover(probability=0.85),
    selection = BestSolutionSelection(),
    termination_criterion =  StoppingByEvaluations(max= 100)
)
algorithm.run()

# Create one random solution
#solution = problem.create_solution()

# Evaluate the solution
#evaluated_solution = problem.evaluate(solution)


In [3]:

def contains_all_ones(milestones, solution):
    for i in range(len(milestones)):
        if milestones[i] == 1 and solution[i] != 1:
            return 0
    return 1

In [12]:
milestones= [0,1,1,0,0]
solution=[1,1,1,1,0]

In [13]:
contains_all_ones(milestones,solution)

1